<a href="https://colab.research.google.com/github/gagao9815/BinSearchEx/blob/master/Homework6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Part1
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Load the data
df = pd.read_csv('dummy_data.csv')

# Create dummy variables for income
income_dummies = pd.get_dummies(df['income'], drop_first=True)
df = pd.concat([df, income_dummies], axis=1)

# Define the model formula
formula = 'time_spent ~ C(income)'

# Fit the model
model = sm.formula.glm(formula, family=sm.families.Binomial(), data=df).fit()

# Print the model summary
print(model.summary())


In [11]:
import pymc as pm


# Preprocess data
median_income = df['income'].median()
df['income_binary'] = (df['income'] > median_income).astype(int)
X = df[['time_spent']].values
y = df['income_binary'].values

# Model setup
with pm.Model() as logistic_model:
    intercept = pm.Normal('Intercept', 0, sigma=20)
    time_spent_coef = pm.Normal('time_spent', 0, sigma=20)
    σ = pm.HalfNormal('σ', sigma=10)

    # Linear model
    likelihood = pm.Bernoulli('y', pm.math.sigmoid(intercept + time_spent_coef * X[:, 0]), observed=y)

    # Sampling
    trace = pm.sample(1000, tune=1000, target_accept=0.95, return_inferencedata=True)


In [12]:
# Transform 'income' into a binary outcome for illustration
median_income = df['income'].median()
df['income_binary'] = (df['income'] > median_income).astype(int)

# Predictor and outcome
X = df['time_spent'].values
y = df['income_binary'].values

with pm.Model() as logistic_model:
    # Priors
    intercept = pm.Normal('Intercept', mu=0, sigma=10)  # Weakly informative prior for the intercept
    time_spent_coef = pm.Normal('time_spent_coef', mu=0, sigma=10)  # Weakly informative prior for the coefficient

    # Linear combination of predictors and logit link function
    logits = intercept + time_spent_coef * X

    # Likelihood (Bernoulli observed outcomes)
    outcome = pm.Bernoulli('outcome', pm.math.sigmoid(logits), observed=y)

    # Sampling
    trace = pm.sample(1000, tune=1000, target_accept=0.95, return_inferencedata=True)


In [16]:
#Part2
from sklearn.linear_model import Ridge, Lasso

# Generate some random data
X = np.random.rand(100, 1)
y = 2*X + np.random.randn(100)

# Define the models
ridge = Ridge(alpha=10)
lasso = Lasso(alpha=0.1)

# Fit the models
ridge.fit(X, y)
lasso.fit(X, y)

# Print the coefficients
print("Ridge coefficients:", ridge.coef_)
print("Lasso coefficients:", lasso.coef_)

# Plot the coefficients
import matplotlib.pyplot as plt

plt.plot(X, y, 'b.')
plt.plot(X, ridge.predict(X), 'r-', label='Ridge')
plt.plot(X, lasso.predict(X), 'g-', label='Lasso')
plt.legend()
plt.show()


#Bayesians do not optimize posterior distributions, they sample from them; but, the posterior distributions are nonetheless 'regularizations' of the likelihood through the prior.

Ridge coefficients: [0.00163166]
Lasso coefficients: [0.]


In [18]:
#Part3
import arviz as az


# Preparing the data
X = df['time_spent'].values[:, None]  # Predictor
y = df['income'].values  # Outcome
n = len(y)

# Model parameters
ν = 5  # Degrees of freedom for the Student's t-distribution, adjust based on your preference
w = np.std(y)  # Scale parameter based on data's standard deviation, adjust as necessary

with pm.Model() as robust_regression_model:
    # Priors
    Intercept = pm.Normal('Intercept', mu=0, sigma=10)
    β = pm.Normal('β', mu=0, sigma=10, shape=X.shape[1])  # Adjust shape based on the number of predictors
    μ = Intercept + pm.math.dot(X, β)

    # Student's t-distribution for the likelihood
    y_obs = pm.StudentT('y_obs', nu=ν, mu=μ, sigma=w, observed=y)

    # Inference
    trace = pm.sample(1000, tune=1000, target_accept=0.95, return_inferencedata=True)

# Optional: Summary of the posterior
summary = az.summary(trace)
print(summary)


              mean      sd   hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  \
Intercept   35.030  10.200   16.574   55.248      0.280    0.198    1343.0   
β[0]       178.828  10.042  160.686  198.025      0.288    0.204    1222.0   

           ess_tail  r_hat  
Intercept    1092.0    1.0  
β[0]         1142.0    1.0  


In [20]:
X = df['time_spent'].values[:, None]  # Predictor
y = df['income'].values  # Outcome
n = len(y)

with pm.Model() as model:
    # Model parameters
    ν = 5  # Degrees of freedom, assuming fixed based on earlier model
    alpha = ν / 2
    beta = ν / 2

    # Priors
    Intercept = pm.Normal('Intercept', mu=0, sigma=10)
    β = pm.Normal('β', mu=0, sigma=10, shape=X.shape[1])  # Adjust shape for the number of predictors
    μ = Intercept + pm.math.dot(X, β)

    # Prior for the precision parameters (lambda_i^(-2))
    λ_inv_squared = pm.Gamma('λ_inv_squared', alpha=alpha, beta=beta, shape=n)
    λ = pm.Deterministic('λ', 1 / pm.math.sqrt(λ_inv_squared))  # Transform to lambda_i

    # Normal likelihood with precision parameter
    y_obs = pm.Normal('y_obs', mu=μ, sigma=1/λ, observed=y)

    # Sample from the posterior
    trace = pm.sample(1000, tune=1000, target_accept=0.95, return_inferencedata=True)

# Posterior analysis to identify outliers
# Lower lambda_i values indicate higher variance and potential outliers
λ_posterior_means = np.mean(trace.posterior['λ'].values, axis=(0, 1))
outlier_threshold = np.percentile(λ_posterior_means, 25)  # Adjust threshold as needed
outliers = np.where(λ_posterior_means < outlier_threshold)[0]

print(f"Potential Outliers (0-indexed): {outliers}")


Potential Outliers (0-indexed): [  0   5  11  20  23  26  33  41  45  48  50  51  52  55  57  61  64  66
  68  70  71  73  75  76  78  86  87  88  90  93  94  97 117 118 121 123
 124 128 132 133 134 135 136 137 138 142 144 150 153 167 170 178 179 184
 188 195 202 205 208 209 216 218 221 224 233 237 239 250 252 255 260 263
 264 270 275 276 280 281 282 287 289 295 297 298 301 304 310 314 326 328
 332 337 339 343 348 351 357 361 362 363 366 369 383 386 387 399 400 405
 406 409 414 423 424 427 431 434 438 439 452 459 463 473 474 475 476 480
 483 488 490 491 493 497 503 506 508 510 518 519 522 525 531 540 542 543
 547 549 551 552 553 554 558 560 563 567 568 577 581 587 588 597 608 610
 623 627 628 630 631 634 636 643 647 652 664 667 668 671 679 681 690 693
 699 704 715 718 720 721 722 724 728 730 734 736 744 745 747 749 763 775
 780 781 783 784 788 790 796 810 813 817 825 831 832 833 835 836 838 839
 841 851 855 860 869 879 883 884 886 887 893 894 898 908 917 919 922 924
 933 937 938 942 94